#**RECOMMENDATION SYSTEM**

In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
class MovieRecommender:
    def __init__(self):

        # Initialize the recommender with empty dictionaries and matrices
        self.movies = {}
        self.genre_matrix = None
        self.similarity_matrix = None
        self.user_item_matrix = None
        self.user_similarity_matrix = None
        self.movie_index = None
        self.user_index = None

    def add_movie(self, movie_id, title, genres):

        # Add a movie to the movies dictionary with its title and genres
        self.movies[movie_id] = {'title': title, 'genres': genres}

    def add_ratings(self, ratings):

        # Add user ratings and create the user-item matrix
        self.ratings = ratings
        self.user_item_matrix = ratings.pivot_table(index='userId', columns='movieId', values='rating').fillna(0)
        self.movie_index = {movie_id: i for i, movie_id in enumerate(self.user_item_matrix.columns)}
        self.user_index = {user_id: i for i, user_id in enumerate(self.user_item_matrix.index)}

    def build_genre_matrix(self):

        # Build a genre matrix where each movie is represented by its genres
        all_genres = sorted(list(set(genre for movie in self.movies.values() for genre in movie['genres'])))
        self.genre_matrix = np.zeros((len(self.movies), len(all_genres)))

        for i, (movie_id, movie) in enumerate(self.movies.items()):
            for genre in movie['genres']:
                j = all_genres.index(genre)
                self.genre_matrix[i, j] = 1

        # Calculate the similarity matrix using cosine similarity between genre vectors
        self.similarity_matrix = cosine_similarity(self.genre_matrix)

    def calculate_user_similarity(self):

        # Calculate user similarity matrix using cosine similarity between user rating vectors
        self.user_similarity_matrix = cosine_similarity(self.user_item_matrix)

    def get_collaborative_recommendations(self, user_id, n=5):

        # Provide collaborative filtering recommendations based on user similarity
        if user_id not in self.user_index:
            return []

        user_index = self.user_index[user_id]
        similar_users = list(enumerate(self.user_similarity_matrix[user_index]))
        similar_users = sorted(similar_users, key=lambda x: x[1], reverse=True)

        # Exclude the user itself and get top n similar users
        similar_users = similar_users[1:n+1]

        recommendations = pd.Series(dtype=float)
        for i, score in similar_users:
            similar_user_id = self.user_item_matrix.index[i]
            similar_user_ratings = self.user_item_matrix.loc[similar_user_id]
            similar_user_ratings = similar_user_ratings[similar_user_ratings > 0]
            recommendations = pd.concat([recommendations, similar_user_ratings])

        recommendations = recommendations.groupby(recommendations.index).mean().sort_values(ascending=False)
        recommended_items = recommendations.index.difference(self.user_item_matrix.loc[user_id][self.user_item_matrix.loc[user_id] > 0].index)

        return [(self.movies[movie_id]['title'], recommendations[movie_id]) for movie_id in recommended_items[:n]]

    def get_content_based_recommendations(self, movie_id, n=5):

        # Provide content-based recommendations based on movie genres
        if movie_id not in self.movies:
            return []

        movie_index = list(self.movies.keys()).index(movie_id)
        similar_movies = list(enumerate(self.similarity_matrix[movie_index]))
        similar_movies = sorted(similar_movies, key=lambda x: x[1], reverse=True)

        # Exclude the movie itself and get top n similar movies
        similar_movies = similar_movies[1:n+1]

        recommendations = [(self.movies[list(self.movies.keys())[i]]['title'], score) for i, score in similar_movies]
        return recommendations

    def build_combined_recommendations(self, user_id, n=5):

        # Combine collaborative and content-based recommendations
        collab_recs = self.get_collaborative_recommendations(user_id, n)
        content_recs = []

        for movie_title, _ in collab_recs:
            movie_id = next((mid for mid, details in self.movies.items() if details['title'] == movie_title), None)
            if movie_id:
                content_recs.extend(self.get_content_based_recommendations(movie_id, n))

        unique_recs = {title: score for title, score in content_recs}
        combined_recs = sorted(unique_recs.items(), key=lambda x: x[1], reverse=True)

        return combined_recs[:n]

Example usage

In [3]:
recommender = MovieRecommender()

Add some movies

In [4]:
recommender.add_movie(1, "The Shawshank Redemption", ["Drama"])
recommender.add_movie(2, "The Godfather", ["Crime", "Drama"])
recommender.add_movie(3, "The Dark Knight", ["Action", "Crime", "Drama"])
recommender.add_movie(4, "Pulp Fiction", ["Crime", "Drama"])
recommender.add_movie(5, "Fight Club", ["Drama"])
recommender.add_movie(6, "Inception", ["Action", "Adventure", "Sci-Fi"])
recommender.add_movie(7, "The Matrix", ["Action", "Sci-Fi"])
recommender.add_movie(8, "Goodfellas", ["Biography", "Crime", "Drama"])
recommender.add_movie(9, "The Silence of the Lambs", ["Crime", "Drama", "Thriller"])
recommender.add_movie(10, "Star Wars: Episode IV - A New Hope", ["Action", "Adventure", "Fantasy"])

Add some ratings

In [5]:
ratings_data = {'userId': [1, 1, 1, 2, 2, 2, 3, 3, 3],
                'movieId': [1, 2, 3, 1, 4, 5, 6, 7, 8],
                'rating': [5, 4, 3, 5, 4, 4, 5, 4, 4]}

ratings = pd.DataFrame(ratings_data)

recommender.add_ratings(ratings)

Build the genre matrix and similarity matrix

In [6]:
recommender.build_genre_matrix()

Calculate user similarity matrix

In [7]:
recommender.calculate_user_similarity()

Get collaborative recommendations for a user

In [8]:
user_id = 1
collab_recommendations = recommender.get_collaborative_recommendations(user_id)
print(f"Collaborative Recommendations for User {user_id}:")
for title, score in collab_recommendations:
    print(f"- {title} (Score: {score:.2f})")

Collaborative Recommendations for User 1:
- Pulp Fiction (Score: 4.00)
- Fight Club (Score: 4.00)
- Inception (Score: 5.00)
- The Matrix (Score: 4.00)
- Goodfellas (Score: 4.00)


Get content-based recommendations for a movie

In [9]:
# The Dark Knight
movie_id = 3
content_recommendations = recommender.get_content_based_recommendations(movie_id)
print(f"\nContent-Based Recommendations for '{recommender.movies[movie_id]['title']}':")
for title, score in content_recommendations:
    print(f"- {title} (Similarity: {score:.2f})")


Content-Based Recommendations for 'The Dark Knight':
- The Godfather (Similarity: 0.82)
- Pulp Fiction (Similarity: 0.82)
- Goodfellas (Similarity: 0.67)
- The Silence of the Lambs (Similarity: 0.67)
- The Shawshank Redemption (Similarity: 0.58)


Get combined recommendations for a user

In [10]:
combined_recommendations = recommender.build_combined_recommendations(user_id)
print(f"\nCombined Recommendations for User {user_id}:")
for title, score in combined_recommendations:
    print(f"- {title} (Score: {score:.2f})")


Combined Recommendations for User 1:
- Fight Club (Score: 1.00)
- Pulp Fiction (Score: 0.82)
- The Godfather (Score: 0.82)
- The Matrix (Score: 0.82)
- Inception (Score: 0.82)
